# Question 1

## Implementation and Architecture

* Implemented a Siamese network to classify speaker voice.
* RNN model with 2 GRU cells.
* GRU
   - units: 512
   - dropout: 0.1
* Batch Normalization
* Dense Layer with 10 output units and relu activation
* L2 normalization
* Flattening layer
* Take output for X_left, X_right and find innerproduct .
* Apply sigmoid over the output inner product.
* Loss - sigmoid cross entropy logits
* Optimizer: Adam
* Learning Rate: 0.0001

## Conclusion
* Tried different networks using LSTM cells, GRU cells, added batch,l2 normalization.
* Accuracy on test set is not great.
* Achieved accuracy around **$67\%$**


In [0]:
import tensorflow as tf
import pickle
import wave, os, glob
import librosa
import numpy as np
import itertools
from random import randint
from random import choice

In [0]:
with open('hw4_trs.pkl', 'rb') as f:
    train_s = pickle.load(f)

In [0]:
def get_stft(sgn):
#   s, sr=librosa.load(filename, sr=None)
  S_sgn=librosa.stft(sgn, n_fft=1024, hop_length=512)
  S_sgn = np.abs(S_sgn).T
#   S_sgn = np.abs(S_sgn)
  return S_sgn

In [0]:
def gen_minibatch(train_s, speaker, size):
  N = 10
  L = size
  sp_s = train_s[speaker*10: (speaker*10) + 10]
  pos_minibatch = []
  for x in range(L):
    curr_sp_i = randint(0, N-1)
    rest_sp_i = randint(0, N-1)
#     print(curr_sp_i, rest_sp_i)
    pos_minibatch.append([sp_s[curr_sp_i], sp_s[rest_sp_i]])
#   list_pos = [[sp_s[i], sp_s[j]] for (i, j) in itertools.combinations(list(range(N)), 2)]
#   pos_minibatch = list_pos[0:L]
  neg_minibatch = []
  for x in range(L):
    curr_sp_i = randint(0, N-1)
#     rest_sp_i = randint(N, len(train_s)-1)
    numbers = list(range(0,speaker*N)) + list(range(speaker*N + 10,len(train_s)))
    rest_sp_i = choice(numbers)
#     print(curr_sp_i, rest_sp_i)
    neg_minibatch.append([sp_s[curr_sp_i], train_s[rest_sp_i]])
  minibatch = []
  minibatch_left = []
  minibatch_right = []
  minibatch_out = []
#   print(len(pos_minibatch))
  for row_p in pos_minibatch:
#     minibatch.append(row_p)
    minibatch_left.append(np.array(row_p[0]))
    minibatch_right.append(np.array(row_p[1]))
    minibatch_out.append(np.array(1))
  for row_n in neg_minibatch:
#     minibatch.append(row_n)
    minibatch_left.append(np.array(row_n[0]))
    minibatch_right.append(np.array(row_n[1]))
    minibatch_out.append(np.array(0))
  return np.array(minibatch_left), np.array(minibatch_right), np.array(minibatch_out)

In [0]:
def stft(train_s):
  train_stft = []
  for sgn in train_s:
    train_stft.append(get_stft(sgn))
  return train_stft


In [0]:
train_stft = stft(train_s)
# l,r,o = gen_minibatch(train_stft, 1, 6)
l2, r2, o2 = gen_minibatch(train_stft, 2, 25)

In [0]:
# # print([t for t in r])

# print('-------------')
# print([t for t in r2])

In [56]:
print(len(train_s[0]))
print((train_stft[11].shape))
# print((sp1_mb[0][0].shape))

16180
(32, 513)


In [57]:
tf.reset_default_graph()
X_left = tf.placeholder(tf.float32, shape=(None, None, 513))
X_right = tf.placeholder(tf.float32, shape=(None, None, 513))
y_actual = tf.placeholder(tf.float32, shape=(None))
dropout = tf.placeholder(tf.float32)
train_phase = tf.placeholder(tf.bool)
layers = []
layer = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(512, 
                                                          kernel_initializer =tf.contrib.layers.variance_scaling_initializer()),
                                                          output_keep_prob = dropout)
# layer = tf.contrib.rnn.LSTMCell(513, forget_bias=1.0)
# layer = tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout)
layers.append(layer)
layer = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(512, 
                                                          kernel_initializer =tf.contrib.layers.variance_scaling_initializer()),
                                                          output_keep_prob = dropout)
# layer = tf.contrib.rnn.LSTMCell(513, forget_bias=1.0)
# layer = tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout)
layers.append(layer)
layer = tf.contrib.rnn.MultiRNNCell(layers)

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [58]:
with tf.variable_scope("model") as scope:
  out_left, _ = tf.nn.dynamic_rnn(layer, X_left, dtype=tf.float32)
#   out_left    = tf.layers.batch_normalization(out_left)
  out_left    = tf.layers.batch_normalization(
        inputs=out_left,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        reuse= False,
        training = train_phase)
#   logits_left = tf.contrib.layers.fully_connected(out_left, 10, activation_fn=tf.nn.tanh)
  logits_left = tf.layers.dense(out_left, 10, kernel_initializer= tf.contrib.layers.variance_scaling_initializer(),activation=tf.nn.tanh)
  logits_left = tf.contrib.layers.flatten(logits_left)
  logits_left = tf.nn.l2_normalize(logits_left, 0)
with tf.variable_scope(scope, reuse=True):
  out_right, _ = tf.nn.dynamic_rnn(layer, X_right, dtype=tf.float32)
#   out_right    = tf.layers.batch_normalization(out_right)
  out_right    = tf.layers.batch_normalization(
        inputs=out_right,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        reuse= False,
        training = train_phase)
#   logits_right = tf.contrib.layers.fully_connected(out_right, 10, activation_fn=tf.nn.tanh)
  logits_right = tf.layers.dense(out_right, 10, kernel_initializer= tf.contrib.layers.variance_scaling_initializer(),activation=tf.nn.tanh)
  logits_right = tf.contrib.layers.flatten(logits_right)
  logits_right = tf.nn.l2_normalize(logits_right, 0)
  
  
print(logits_right.shape)  
print(tf.multiply(logits_left,logits_right).shape)
innerpd = tf.reduce_sum(tf.multiply(logits_left,logits_right), axis = 1, keepdims=True, name = 'innerpd')
# innerpd = tf.reduce_sum(tf.multiply(tf.reshape(logits_left, [-1]),tf.reshape(logits_right, [-1])), axis = 1, keepdims=True, name = 'innerpd')
# print(innerpd.shape)
# innerpd = tf.reduce_sum(innerpd, 0)
print("reshaped :", innerpd.shape)

sigmoid_out = tf.nn.sigmoid(innerpd,name = 'sigmoid_out')

print(type(sigmoid_out))
print(sigmoid_out.shape)
# gradOptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# train = gradOptimizer.minimize(loss_siamese)

Instructions for updating:
Use keras.layers.dense instead.
(?, ?)
(?, ?)
reshaped : (?, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1)


In [0]:
# w = tf.reshape(logits_left, -1)
# print(w.shape)

In [0]:
# loss=tf.losses.mean_squared_error(labels=y_actual, predictions=logits)

# train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [0]:
# loss  = tf.reduce_sum(-(y_actual)*tf.log(sigmoid_out+ (10**-6)) - (1 - y_actual) * tf.log(1 - sigmoid_out + (10**-6)))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels= y_actual , logits= sigmoid_out))
# loss = -(y_actual)*tf.log(sigmoid_out+ (10**-6)) - (1 - y_actual) * tf.log(1 - sigmoid_out + (10**-6))
train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

correct_pred = tf.equal(sigmoid_out, y_actual)
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(sigmoid_out),y_actual),tf.float32))

In [0]:
# LOG_DIR = '/tmp/log'
# get_ipython().system_raw(
#     'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#     .format(LOG_DIR)
# )
# # ! curl http://localhost:6006

# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-386.zip
# ! unzip ngrok-stable-windows-386.zip

# get_ipython().system_raw('./ngrok http 6006 &')

# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
    

In [0]:
sess = tf.Session()
init = tf.global_variables_initializer()
saver=tf.train.Saver()
sess.run(init)
# train_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

In [0]:
batch_l = 50
left = []
right = []
out = []
for epoch in range(10):
  loss_print = 0
  for i in range(0,50):
    left, right, out = gen_minibatch(train_stft, i, batch_l)
    loss1, t, inpd, lef, rig = sess.run([loss, train_step, innerpd,logits_left, logits_right], {X_left: left, X_right: right, y_actual: out, dropout: 0.9, train_phase: True})
#     print('Inner product :', inpd)
#     print('Left ', lef[0].shape)
#     print('Right', rig.shape)
#     print(loss1)
  loss_print += loss1
#   print('Loss at epoch',epoch+1, 'is: ', loss_print)

# Training Phase

In [64]:
# print([x.name for x in tf.global_variables()])

['model/rnn/multi_rnn_cell/cell_0/gru_cell/gates/kernel:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/gates/bias:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/candidate/kernel:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/candidate/bias:0', 'model/rnn/multi_rnn_cell/cell_1/gru_cell/gates/kernel:0', 'model/rnn/multi_rnn_cell/cell_1/gru_cell/gates/bias:0', 'model/rnn/multi_rnn_cell/cell_1/gru_cell/candidate/kernel:0', 'model/rnn/multi_rnn_cell/cell_1/gru_cell/candidate/bias:0', 'model/batch_normalization/gamma:0', 'model/batch_normalization/beta:0', 'model/batch_normalization/moving_mean:0', 'model/batch_normalization/moving_variance:0', 'model/dense/kernel:0', 'model/dense/bias:0', 'beta1_power:0', 'beta2_power:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/gates/kernel/Adam:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/gates/kernel/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/gates/bias/Adam:0', 'model/rnn/multi_rnn_cell/cell_0/gru_cell/gates/bias/Adam_1:0', 'model/rnn/multi_rnn

In [0]:
# l = [z for z in left]
# print(l[0][0,0])
# r = [y for y in right]
# print(r)

# print(sum(out==1))

['model/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/kernel:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/bias:0', 'model/fully_connected/weights:0', 'model/fully_connected/biases:0', 'beta1_power:0', 'beta2_power:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel/Adam:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias/Adam:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/kernel/Adam:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/kernel/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/bias/Adam:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/bias/Adam_1:0', 'model/fully_connected/weights/Adam:0', 'model/fully_connected/weights/Adam_1:0', 'model/fully_connected/biases/Adam:0', 'model/fully_connected/biases/Adam_1:0']

['model/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/kernel:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/bias:0', 'model/fully_connected/weights:0', 'model/fully_connected/biases:0', 'model/fully_connected_1/weights:0', 'model/fully_connected_1/biases:0', 'beta1_power:0', 'beta2_power:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel/Adam:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias/Adam:0', 'model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/kernel/Adam:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/kernel/Adam_1:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/bias/Adam:0', 'model/rnn/multi_rnn_cell/cell_1/lstm_cell/bias/Adam_1:0', 'model/fully_connected/weights/Adam:0', 'model/fully_connected/weights/Adam_1:0', 'model/fully_connected/biases/Adam:0', 'model/fully_connected/biases/Adam_1:0', 'model/fully_connected_1/weights/Adam:0', 'model/fully_connected_1/weights/Adam_1:0', 'model/fully_connected_1/biases/Adam:0', 'model/fully_connected_1/biases/Adam_1:0']

In [0]:
## Test phase

with open('hw4_tes.pkl', 'rb') as f:
    test_s = pickle.load(f)
    
test_stft = stft(test_s)  



In [104]:
l2, r2, o2 = gen_minibatch(train_stft, 2, 25)
# print(l2)
acc = 0
for i in range(0,20):
  l,r,o = gen_minibatch(test_stft, i, 20)
  test_accuracy,s_out = sess.run([accuracy,sigmoid_out], feed_dict={X_left: l, X_right:r, y_actual: o, dropout:1.0, train_phase: False}) 
  acc_t = sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.reshape(s_out, [-1])),o),tf.float32)))
#   print(acc_t)
  acc += acc_t
  
acc_final = acc/(20)  
print('\nAccuracy on test set:', acc_final)
# test_accuracy = sess.run(sigmoid_out, feed_dict={X_left: l2, X_right:r2, y_actual: o2, dropout:1.0})
# print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.672
